## DETERMINING HABITABLE ZONES ##

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from astropy import constants as const
from astropy import units as u

# DEFYNING CONSTANTS

In [ ]:
# Definiendo constantes
G = const.G.to(u.cm**3/(u.g*u.s**2)) # Constante Gravitacional en cm^3/(g*s^2)
Msol = const.M_sun.to(u.g) # Masa solar en gramos
R_sol = const.R_sun.to(u.cm)  # Radio del sol en centímetros

In [ ]:
# Read the CSV file with space as the delimiter
exoplanet_data = pd.read_csv('exoplanet_data.csv', sep=',', comment='#')
exoplanet_data

## Calculando zonas de habitabilidad 

Basándonos en el modelo de  (Kopparapu,2013), el cual considera planetas rocosos que tienen una masa menor a $5M_{\oplus}$,donde se asumen atmósferas dominadas por $H_2O$ para el límite interno y $CO{-1}_2$ para el límite externo. 
Este modelo es un modelo unidimensional, en donde se asume la atmósfera como capas, apiladas unas sobre otras. En el modelo planteado por los autores, se ignoran factores como las nubes y se concentran en como el calor se mueve a través de la atmósfera por radiación(proveniente del sol) y la convección, producto del aumento del aire caliente. 
Para este modelo se tuvieron en cuenta los siguientes casos: 1. Toman un planeta fijo y estudian el efecto de los gases no condensables($N_2$) en los límites de la ZH, variando la presión parcial de $N_2$; 2 Fijar una presión debida al $N_2$ de 0.01 bar para distintas masas de planetas entre el intervalo ya mencionado, con el fin de estudiar el efecto de la gravedad por sí solo, y 3 la presión del $N_2$ se ajusta de acuerdo al tamaño del radio del planeta y verificar el efecto del tamaño del planeta en la abundancia volátil.



# MÉTODO 
En el artículo mencionado, se tomaron temperaturas efectivas entre $2500 k\leq T_eff \leq 7200 k$. Para esta primera parte se tomarán los planetas dentro del rango de masa y se tomarán únicamente aquellos planetas cuya estrella sea de secuencia principal.

In [ ]:
# Obtaining plaets with Mass between 0.4 to 5 and then main sequence stars 
kopparu = exoplanet_data[
    (exoplanet_data['Planet_mass[Earth]'] >= 0.4) & 
    (exoplanet_data['Planet_mass[Earth]'] <= 5)
]

kopparu

In [ ]:
main_stars = []
for index, planet in kopparu.iterrows():
    spec_type = str(planet['Spectral_type'])
    if 'V' in spec_type and 'I' not in spec_type:
        main_stars.append(planet)

# Convert the list of rows back to a DataFrame
main_stars_df = pd.DataFrame(main_stars)
main_stars_kopparu = kopparu[kopparu.index.isin(main_stars_df.index)]
main_stars_kopparu['Stellar_effective_temperature']

Para conocer las distancias de los límites de la zona habitable para cada tipo de estrella, se considera la siguiente ecuación paramétrica
$ S_{eff} = S_{eff\odot } + aT_{*} + bT_{*}^2 + cT_{*}^3 + dT_{*}^4$
donde $T_{*} = T_{eff}-5780k.$
Para ello, se toman los parámetros del código proporcionado por kopparu entre 0.1M y 5 M % No sé si entonces añadir planetas de 0.1 porque me tardaría muchooooo

In [ ]:
hzdat = open('HZs.csv', 'w')
# Creating a function for the effective flux 
def Kopparapu_flux (a, b, c, d, t, S_eff_sun):

    return S_eff_sun + a*t + b*t**2 + c * t**3 + d * t**4

In [ ]:

# S_eff_sun = [Recent_venus, Runaway_Greenhouse,Maximum_greenhouse, Early_Mars, 0.1M Runaway_Greenhouse, 5M Runwa]
S_eff_sun = [1.776,1.107, 0.356, 0.320, 1.188, 0.99]
T_eff = np.array(main_stars_kopparu['Stellar_effective_temperature'])
S_eff = np.zeros_like(T_eff)
T = T_eff - 5780 # Se toma como temperatura del sol 5780k
#Calculating coefficients 
a = [2.136e-4, 1.332e-4, 6.171e-5, 5.547e-5, 1.433e-4, 1.209e-4]
b = [2.533e-8, 1.580e-8, 1.698e-9, 1.526e-9, 1.707e-8, 1.404e-8]
c = [-1.332e-11, -8.308e-12, -3.198e-12, -2.874e-12, -8.968e-12, -7.418e-12]
d = [-3.097e-15, -1.931e-15, -5.575e-16, -5.011e-16, -2.084e-15, -1.713e-15]

T

In [ ]:
# Definiendo las variables para los límites 
recentVenus = []
runawayGreenhouse = []
maxGreenhouse = []
earlyMars = []
fivemeRunaway = []
tenthmeRunaway = []

In [ ]:
# Calculating the effective flux for each limit 
for t in T :
    for i in range(len(a)):
        S_eff[i] = Kopparapu_flux (a[i], b[i], c[i], d[i], t, S_eff_sun[i])
        print(S_eff[i])
    
    recentVenus.append(S_eff[0])
    runawayGreenhouse.append(S_eff[1])
    maxGreenhouse.append(S_eff[2])
    earlyMars.append(S_eff[3])
    fivemeRunaway.append(S_eff[4])
    tenthmeRunaway.append(S_eff[5])

In [ ]:
print(np.size(runawayGreenhouse))
fivemeRunaway

In [ ]:
#for i in range(len(T_eff)):
#    hzdat.write(
#        f'   {T_eff.iloc[i]:6.0f}      '
#       f'{S_eff[i][0]:6.6E}      '  # First value in the i-th sublist
#       f'{S_eff[i][1]:6.6E}     '
#       f'{S_eff[i][2]:6.6E}   '
#        f'{S_eff[i][3]:6.6E} '
#       f'{S_eff[i][4]:6.6E} '
#       f'{S_eff[i][5]:6.6E}  \n'

In [ ]:
# Graficando Temperatura vs  S_eff

plt.scatter(recentVenus, T_eff)
plt.scatter(runawayGreenhouse, T_eff, color = 'red')
plt.scatter(maxGreenhouse, T_eff, color = 'orange')
plt.scatter(earlyMars, T_eff, color = 'brown')
plt.scatter(fivemeRunaway, T_eff, color = 'green')
plt.scatter(tenthmeRunaway,T_eff, color= "black")
plt.plot ()
plt.gca().invert_xaxis()
plt.grid(True)
plt.xlabel(r'$S_{eff}$')
plt.ylabel(r'$T_{eff}(K)')
plt.show()

 Ahora bien para calcular las distancias de los límites de la HZ se utilza la siguiente ecuación

$d=\sqrt{\frac{L/L_{\odot}}{S_{eff}}} AU$
Ahora bien, la luminosidad esta dada por, donde $R_{*}$ es el radio de la estrella y T su temperatura, $R_{\odot}$ el radio del sol y $T_{\odot}= 5780$la temperatura del sol. $\sigma$ es la constante de Boltzmann 
$L = 4\pi  R_{*}^{2} \sigma  T^{4}$   
Entonces al remplazar $L$ y $L_{\odot }$ en la ecuación de $d$. 
$d =\sqrt{\frac{4\pi R_{*}^{2}\sigma  T^{4}/ 4\pi R_{\odot}^{2}\sigma T_{\odot}^{4}}{S_{eff}}} =\sqrt{\frac{R_{*}^{2} T^{4}/R_{\odot }^{2}  T_{\odot}^{4} }{S_eff}} AU$

In [ ]:
T_sun = 5780 
# Calculating distances 
def distancia(T_eff,R, S_eff):
    d = np.sqrt((R**2*T_eff/ (T_sun**4))/S_eff)
    return d #Los radios solares ya estan en radios solares 

In [ ]:
R = np.array(main_stars_kopparu['Stellar_radius'])
d = np.zeros_like(R)
d_recentVenus = []
d_runawayGreenhouse = []
d_maxGreenhouse = []
d_earlyMars = []
d_fivemeRunaway = []
d_tenthmeRunaway =[]
for T_star in T_eff:
    for j in range(len(a)):
        d[j] = distancia(T_star,R[j],S_eff[j])
        #print(d[j])
    d_recentVenus.append(d[0])
    d_runawayGreenhouse.append(d[1])
    d_maxGreenhouse.append(d[2])
    d_earlyMars.append(d[3])
    d_fivemeRunaway.append(d[4])
    d_tenthmeRunaway.append(d[5])
d_recentVenus

In [ ]:
print(np.size(d))

In [ ]:
# Graficando Temperatura vs distancias HZ

plt.scatter(d_recentVenus, T_eff)
plt.scatter(d_runawayGreenhouse, T_eff, color = 'red')
plt.scatter(d_maxGreenhouse, T_eff, color = 'orange')
plt.scatter(d_earlyMars, T_eff, color = 'brown')
plt.scatter(d_fivemeRunaway, T_eff, color = 'green')
plt.scatter(d_tenthmeRunaway,T_eff, color= "black")
plt.plot ()
#plt.gca().invert_xaxis()
plt.grid(True)
plt.xlabel(r'$d (AU$)')
plt.ylabel(r'$T_{eff}(K)$')
plt.show()

Después de hallar los límites de las zonas habitables, procedemos a ubicar a los planetas que se encuentren dentro de esta zona. Para ello, calcularemos la distancia que hay entre el planeta y su estrella anfitriona.
Sabiendo que la eccentricidad se define como :
$e=\frac{c}{a}$
donde a representa el semi-eje  mayor y c la distancia que hay entre el objeto y su centro y/o foco(estrella). 
Sabiendo que 
* e = 0 -> órbita circular
* e $\epsilon$ [0,1] -> órbita elíptica
* e > 1 -> hipérbola
Entonces, los planetas cuya órbita tenga eccentricidad $e=0$, entonces $a = c$. Si $ e\epsilon\bigr]0;1\bigr]$, entonces $c = ae$.    

In [ ]:
Semi_major = main_stars_kopparu['Orbit_semimajor_axis'].dropna()
main_stars_kopparu = kopparu[kopparu.index.isin(main_stars_df.index)]
Planets = main_stars_kopparu[main_stars_kopparu.index.isin(Semi_major.index)]
e = np.array(Planets['Orbit_eccentricity'])
a_m = np.array(Semi_major)
print(np.size(e))

In [ ]:
#Calculating distances 
c = []
for i in range(len(e)):
    if e[i] == 0 :
        c.append(a_m[i])
    elif e[i] > 0 and e[i] <=1:
        c.append(a_m[i]*e[i])
c 